---

<a href="https://github.com/rraadd88/roux/blob/master/examples/roux_lib_df.ipynb"><img align="right" style="float:right;" src="https://img.shields.io/badge/-source-cccccc?style=flat-square"></a>

## ⌗ Tables/Dataframes.

In [1]:
# import helper functions
from roux.global_imports import *

0it [00:00, ?it/s]

INFO:root:Python implementation: CPython
Python version       : 3.7.13
IPython version      : 7.31.1
scipy     : 1.7.3
matplotlib: 3.5.1
logging   : 0.5.1.2
pandas    : 1.3.5
sys       : 3.7.13 (default, Mar 29 2022, 02:18:16) 
[GCC 7.5.0]
numpy     : 1.18.1
seaborn   : 0.11.2
tqdm      : 4.64.1
re        : 2.2.1



INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


### Helper functions for basic data validations 

In [2]:
## demo data
import seaborn as sns
df1=sns.load_dataset('iris')

In [3]:
# .rd (roux data) attributes
## validate no missing values in the table
assert df1.rd.validate_no_na()
## validate no duplicates in the table
df1.rd.validate_no_dups()

False

### Helper functions for checking duplicates in a table

In [4]:
df1.rd.check_dups()

INFO:root:duplicate rows: 1.3% (2/150)


,sepal_length,sepal_width,petal_length,petal_width,species
101,5.8,2.7,5.1,1.9,virginica
142,5.8,2.7,5.1,1.9,virginica


### Helper functions for logging changes in the dataframe shapes

In [5]:
_=df1.log.drop_duplicates()

INFO:root:drop_duplicates: shape changed: (150, 5)->(149, 5), width constant


In [6]:
## within pipes
_=(df1
   .log.drop_duplicates()
   .log('sepal_length')
   .log('sepal_length',groupby='species')
  )

INFO:root:drop_duplicates: shape changed: (150, 5)->(149, 5), width constant
INFO:root:shape = (149, 5) nunique: sepal_length = 35
INFO:root:shape = (149, 5) by "species", nunique "sepal_length": setosa = 15; versicolor = 21; virginica = 21


### Helper functions to filter dataframe using a dictionary

In [7]:
_=df1.rd.filter_rows({'species':'setosa'})

INFO:root:(150, 5)
INFO:root:(50, 5)


### Helper functions to merge tables while validating the changes in shapes  

In [8]:
df2=df1.groupby('species').head(1)

In [9]:
df1.log.merge(right=df2,
              how='inner',
              on='species',
              validate='m:1',
             validate_equal_length=True,
             # validate_no_decrease_length=True,
             ).head(1)

INFO:root:merge: shape changed: (150, 5)->(150, 9), length constant


,sepal_length_x,sepal_width_x,petal_length_x,petal_width_x,species,sepal_length_y,sepal_width_y,petal_length_y,petal_width_y
0,5.1,3.5,1.4,0.2,setosa,5.1,3.5,1.4,0.2


#### Documentation
[`roux.lib.df`](https://github.com/rraadd88/roux#module-roux.lib.df)
[`roux.lib.dfs`](https://github.com/rraadd88/roux#module-roux.lib.dfs)